In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")



In [2]:
import sys
print(sys.executable)

/root/anaconda3/bin/python


In [3]:
data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")
data.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


스케일링 하는데 fiberid 를 어떻게 다루는것이 좋을지에 대해 비교해보자
* 일단 fivberid는 수치비교가 무의미한 데이터이기 때문에 수학적인 계산이 들어가는 std스케일 등은 의미가 없다
* 따라서 다음 두가지 방식으로 비교를 한다
        -> fiberid를 그대로두는 방식
        -> fiberid를 maxabs로 스케일링 하는방식
* 이때 fiberid를 제외한 나머지 변수들은 std로 스케일링 하였다.

In [4]:
data_for_scaling = data[data.columns[3:]]
data_for_scaling.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


* fiberid를 그대로 두고 스케일링 하는방식

In [5]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
# type과 id, fiberid 를 제외하고 학습
std_scaler.fit(data_for_scaling)
# 학습후 변환
train_stdscaler = std_scaler.transform(data_for_scaling)
train_stdscaler
# 학습후 변환한 데이터를 다시 원래 데이터로 만들기
data_stdscaler = pd.DataFrame(train_stdscaler, columns = data_for_scaling.columns)
data_stdscaler_rawfiberID = pd.concat([data[['id','type', 'fiberID']], data_stdscaler], axis=1)
data_stdscaler_rawfiberID.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,0.002522,0.017736,0.022913,0.026865,0.028350,0.002812,0.003432,...,0.000844,0.019287,0.030818,0.023734,0.019085,0.021572,0.018062,0.023754,0.025371,0.029607
1,1,QSO,788,0.002373,0.013079,0.017916,0.022808,0.022707,0.002641,0.002612,...,-0.000604,0.015060,0.024765,0.020265,0.020656,0.011300,0.013691,0.019199,0.022876,0.024857
2,2,QSO,427,0.002071,-0.012530,-0.013542,-0.011566,-0.008257,0.001755,-0.002722,...,-0.005030,-0.011112,-0.018435,-0.006467,-0.006246,-0.018749,-0.011420,-0.011496,-0.007869,-0.003743
3,3,QSO,864,0.002319,0.008782,0.012001,0.015437,0.014545,0.002442,0.001661,...,-0.001778,0.009973,0.015467,0.014165,0.012306,0.005394,0.009011,0.012750,0.015747,0.017680
4,4,STAR_RED_DWARF,612,0.002796,0.028203,0.024150,0.012532,0.005864,0.003560,0.006081,...,0.005094,0.025735,0.032826,0.013198,0.004744,0.038971,0.028465,0.024583,0.013683,0.008874


* fiberid를 ma 방식으로 스케일링 하는방식

In [6]:
from sklearn.preprocessing import MaxAbsScaler
ma_scaler = MaxAbsScaler()

ma_scaler.fit(data[['fiberID']])
fiberid_scale_matrix = ma_scaler.transform(data[['fiberID']])
fiberid_mascaled = pd.DataFrame(fiberid_scale_matrix, columns = ['fiberID'])
# fiberid_mascaled.head()

data_stdscaler_scaledfiberID = pd.concat([data[['id','type']],fiberid_mascaled, data_stdscaler], axis=1)
data_stdscaler_scaledfiberID.head()


,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,0.601,0.002522,0.017736,0.022913,0.026865,0.028350,0.002812,0.003432,...,0.000844,0.019287,0.030818,0.023734,0.019085,0.021572,0.018062,0.023754,0.025371,0.029607
1,1,QSO,0.788,0.002373,0.013079,0.017916,0.022808,0.022707,0.002641,0.002612,...,-0.000604,0.015060,0.024765,0.020265,0.020656,0.011300,0.013691,0.019199,0.022876,0.024857
2,2,QSO,0.427,0.002071,-0.012530,-0.013542,-0.011566,-0.008257,0.001755,-0.002722,...,-0.005030,-0.011112,-0.018435,-0.006467,-0.006246,-0.018749,-0.011420,-0.011496,-0.007869,-0.003743
3,3,QSO,0.864,0.002319,0.008782,0.012001,0.015437,0.014545,0.002442,0.001661,...,-0.001778,0.009973,0.015467,0.014165,0.012306,0.005394,0.009011,0.012750,0.015747,0.017680
4,4,STAR_RED_DWARF,0.612,0.002796,0.028203,0.024150,0.012532,0.005864,0.003560,0.006081,...,0.005094,0.025735,0.032826,0.013198,0.004744,0.038971,0.028465,0.024583,0.013683,0.008874


* 위와 동일한 방식으로 test data도 스케일링 하여 만들어주자

In [7]:
testdata_for_scaling = test_data[test_data.columns[2:]]
testdata_for_scaling.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,23.817399,22.508963,20.981106,18.517316,17.076079,25.053890,23.167848,21.335901,18.835858,17.434384,22.246697,22.796239,21.195315,18.584486,17.154284,25.391534,22.499435,21.011918,18.499341,17.091474
1,22.806983,21.937111,20.335770,20.000512,19.527369,22.498565,22.186000,20.618879,20.301204,20.030078,21.729831,21.837511,20.196128,19.967204,19.683671,22.475338,21.853442,20.173169,19.796757,19.567372
2,21.024250,19.235669,18.304061,17.808608,17.380113,21.205546,19.439533,18.344433,17.909690,17.538164,20.722629,18.710223,17.611851,17.158519,16.843986,20.579314,18.653338,17.562108,17.120529,16.708748
3,20.503424,20.286261,20.197204,20.162419,20.059832,20.976132,20.611498,20.567262,20.479318,20.277472,20.329269,20.385262,20.129157,20.206574,20.212342,20.479879,20.280943,20.150499,20.206221,20.092909
4,24.244851,22.668237,21.239333,19.284777,18.235939,25.681860,22.935289,21.642456,19.624926,18.591749,22.308298,22.957496,21.285033,19.299120,18.307526,25.489360,22.857290,21.191862,19.237964,18.280368


* fiberID를 그대로 둔 상태에서의 test 데이터 생성

In [8]:
test_stdscaler = std_scaler.transform(testdata_for_scaling)
# scale된 테스트 데이터를 원래 모양으로 복구
testdata_stdscaler = pd.DataFrame(test_stdscaler, columns = testdata_for_scaling.columns)
testdata_stdscaler_rawfiberID = pd.concat([test_data[['id', 'fiberID']], testdata_stdscaler], axis=1)
testdata_stdscaler_rawfiberID.head()

,id,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,199991,251,0.002574,0.024666,0.020294,0.004063,-0.004748,0.003405,0.005466,0.024447,...,0.000518,0.028127,0.027908,0.006161,-0.003819,0.043177,0.024455,0.021125,0.006151,-0.000732
1,199992,386,0.002489,0.020986,0.015217,0.016781,0.015063,0.002792,0.004155,0.016484,...,-0.000137,0.021916,0.017633,0.015649,0.013907,0.019333,0.020461,0.014865,0.016041,0.017789
2,199993,232,0.002339,0.003605,-0.000764,-0.002014,-0.002290,0.002482,0.000490,-0.008774,...,-0.001413,0.001659,-0.008944,-0.003624,-0.005993,0.003829,0.000674,-0.004623,-0.004359,-0.003595
3,199994,557,0.002295,0.010365,0.014127,0.018169,0.019366,0.002427,0.002054,0.015911,...,-0.001911,0.012509,0.016944,0.017292,0.017612,0.003016,0.010738,0.014695,0.019163,0.021720
4,199995,75,0.002610,0.025690,0.022325,0.010644,0.004626,0.003555,0.005155,0.027851,...,0.000596,0.029171,0.028831,0.011065,0.004263,0.043977,0.026668,0.022468,0.011782,0.008162


* fiberID를 scaling한 상태에서의 test 데이터 생성

In [9]:
testfiberid_scale_matrix = ma_scaler.transform(test_data[['fiberID']])
testfiberid_mascaled = pd.DataFrame(testfiberid_scale_matrix, columns = ['fiberID'])
# fiberid_mascaled.head()

testdata_stdscaler_scaledfiberID = pd.concat([test_data[['id',]],testfiberid_mascaled, testdata_stdscaler], axis=1)
testdata_stdscaler_scaledfiberID.head()

,id,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,199991,0.251,0.002574,0.024666,0.020294,0.004063,-0.004748,0.003405,0.005466,0.024447,...,0.000518,0.028127,0.027908,0.006161,-0.003819,0.043177,0.024455,0.021125,0.006151,-0.000732
1,199992,0.386,0.002489,0.020986,0.015217,0.016781,0.015063,0.002792,0.004155,0.016484,...,-0.000137,0.021916,0.017633,0.015649,0.013907,0.019333,0.020461,0.014865,0.016041,0.017789
2,199993,0.232,0.002339,0.003605,-0.000764,-0.002014,-0.002290,0.002482,0.000490,-0.008774,...,-0.001413,0.001659,-0.008944,-0.003624,-0.005993,0.003829,0.000674,-0.004623,-0.004359,-0.003595
3,199994,0.557,0.002295,0.010365,0.014127,0.018169,0.019366,0.002427,0.002054,0.015911,...,-0.001911,0.012509,0.016944,0.017292,0.017612,0.003016,0.010738,0.014695,0.019163,0.021720
4,199995,0.075,0.002610,0.025690,0.022325,0.010644,0.004626,0.003555,0.005155,0.027851,...,0.000596,0.029171,0.028831,0.011065,0.004263,0.043977,0.026668,0.022468,0.011782,0.008162


In [10]:
data_stdscaler_rawfiberID.to_csv('./scaledtraindata/train_std_scaled_with_rawfiberid.csv', sep=',', index = False)
data_stdscaler_scaledfiberID.to_csv('./scaledtraindata/train_mm_scaled_with_scaledfiberid.csv', sep=',', index = False)

In [11]:
from lightgbm import LGBMClassifier
import pickle

# raw 에 사용할 모델
lgbm_model_stdscaler_rawfiberID = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=2000, max_depth=15,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)

# scaled된 fiberID에 사용할 모델
lgbm_model_stdscaler_scaledfiberID = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=2000, max_depth=15,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)

* fiberID의 스케일링에 따라 각 모델을 학습시켰고 학습모델을 바로 저장하였다

In [12]:
X_raw = data_stdscaler_rawfiberID[data.columns[2:]]
y_raw = data_stdscaler_rawfiberID['type']

In [31]:
# fiberid가 raw한 모델을 학습
lgbm_model_stdscaler_rawfiberID.fit(X_raw, y_raw)
pickle.dump(lgbm_model_stdscaler_rawfiberID, open('./models/lgbm_stdscaler_rawfiberID.sav', 'wb'))


In [34]:
X_scaled = data_stdscaler_scaledfiberID[data.columns[2:]]
y_scaled = data_stdscaler_scaledfiberID['type']

In [36]:
# fiberid가 scaled된 모델을 학습
lgbm_model_stdscaler_scaledfiberID.fit(X_scaled, y_scaled)
pickle.dump(lgbm_model_stdscaler_scaledfiberID, open('./models/lgbm_stdscaler_scaledfiberID.sav', 'wb'))

학습된 모델을 이용해 test data 확률 예측 및 결과 저장.

In [12]:
id_for_index = test_data['id']
test_raw = testdata_stdscaler_rawfiberID[test_data.columns[1:]]
test_scaled = testdata_stdscaler_scaledfiberID[test_data.columns[1:]]

sample = pd.read_csv("./data/sample_submission.csv")
col_order = sample.columns[1:]

In [13]:
# 저장된 모델 불러오기
model1 = pickle.load(open('./models/lgbm_stdscaler_rawfiberID.sav', 'rb'))
model2 = pickle.load(open('./models/lgbm_stdscaler_scaledfiberID.sav', 'rb'))

In [14]:
test_raw.head(5)
len(test_raw)


10009

In [15]:
model1

LGBMClassifier(bagging_fraction=0.9, boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, feature_fraction=0.9,
        importance_type='split', learning_rate=0.1, max_depth=15,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=2000, n_jobs=-1, num_leaves=10, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.2, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# 확률 행렬 도출

predictions_raw = model1.predict_proba(X = test_raw)
print(predictions_raw)

In [31]:
len(predictions_raw)

199991

In [22]:
result_raw = pd.DataFrame(data=predictions_raw, index=id_for_index, columns=model1.classes_)
result_raw = result_raw[col_order]
result_raw.to_csv('./result/lgbm_stdscaler_rawfiberID.csv', sep=',')

ValueError: Shape of passed values is (199991, 19), indices imply (10009, 19)

In [ ]:
predictions_scaled = model2.predict_proba(X = test_scaled)

In [ ]:
result_scaled = pd.DataFrame(data=predictions_raw, index=id_for_index, columns=model2.classes_)
result_scaled = result_scaled[col_order]
rseult_scaled.to_csv('./result/lgbm_stdscaler_scaledfiberID.csv', sep=',')